In [15]:
import tensorflow as tf
import numpy as np

K = 5
NUM_STEP = 3
T = 8
BATCH_SIZE = 8
NUM_ACTIONS = 18

In [16]:
import tensorflow as tf
import numpy as np

class layer():

    #Inputs: 
    #   k_screens : tensor, [None, K, 33600]
    #Outputs:
    #   dense : tensor, [None, 1024]
    #   cnn_shapes : python list, shapes of each convolution layers
    def conv_encoder(k_screens):
        k_screens = tf.reshape(k_screens, tf.stack([tf.shape(k_screens)[0], K, 210, 160]))
        NHWC_k_screens = tf.transpose(k_screens, [0, 2, 3, 1])

        cnn_shapes = []
        with tf.variable_scope("encoder"):
            with tf.variable_scope("cnn_layers", reuse=tf.AUTO_REUSE):
                cnn_shapes.append(NHWC_k_screens.get_shape().as_list())
                w1 = tf.get_variable("conv1_weights", [8, 8, 5, 16], initializer=tf.random_normal_initializer())
                conv1 = tf.nn.relu(tf.nn.conv2d(NHWC_k_screens, w1, strides=[1, 2, 2, 1], padding='SAME'))
                cnn_shapes.append(conv1.get_shape().as_list())

                w2 = tf.get_variable("conv2_weights", [6, 6, 16, 32], initializer=tf.random_normal_initializer())
                conv2 = tf.nn.relu(tf.nn.conv2d(conv1, w2, strides=[1, 2, 2, 1], padding='SAME'))
                cnn_shapes.append(conv2.get_shape().as_list())

                w3 = tf.get_variable("conv3_weights", [6, 6, 32, 64], initializer=tf.random_normal_initializer())
                conv3 = tf.nn.relu(tf.nn.conv2d(conv2, w3, strides=[1, 2, 2, 1], padding='SAME'))
                cnn_shapes.append(conv3.get_shape().as_list())

                w4 = tf.get_variable("conv4_weights", [4, 4, 64, 64], initializer=tf.random_normal_initializer())
                conv4 = tf.nn.relu(tf.nn.conv2d(conv3, w4, strides=[1, 2, 2, 1], padding='SAME'))
                cnn_shapes.append(conv4.get_shape().as_list())

            with tf.variable_scope("flatten_layers", reuse=tf.AUTO_REUSE):
                flatten = tf.contrib.layers.flatten(inputs = conv4)

                w = tf.get_variable("dense_weights", [14*10*64, 1024], initializer=tf.random_normal_initializer())
                b = tf.get_variable("dense_bias", [1024], initializer=tf.random_normal_initializer())
                dense = tf.nn.relu(tf.matmul(flatten, w) + b)
        return dense, cnn_shapes


    #Inputs:
    #   encode : tensor, [None, 1024]
    #   act : tensor, [None, 18]
    #Outputs:
    #   pred_encoder : tensor, [None, 1024]
    def action_transform(encode, act):
        with tf.variable_scope("action_transform", reuse=tf.AUTO_REUSE):
            w1 = tf.get_variable("encoder_factor_weights", [1024,2048], initializer=tf.random_normal_initializer())
            b1 = tf.get_variable("encoder_factor_bias", [2048], initializer=tf.random_normal_initializer())
            encode_factor = tf.matmul(encode, w1) + b1
            hidden = tf.cast(encode_factor, tf.int32, name = "hidden")

            w2 = tf.get_variable("act_emb_weights", [18,2048], initializer=tf.random_normal_initializer())
            b2 = tf.get_variable("act_emb_bias", [2048], initializer=tf.random_normal_initializer())
            act_emb = tf.matmul(act, w2) + b2

            decode_factor = tf.multiply(encode_factor, act_emb)

            w3 = tf.get_variable("pred_encode_weights", [2048,1024], initializer=tf.random_normal_initializer())
            b3 = tf.get_variable("pred_encode_bias", [1024], initializer=tf.random_normal_initializer())
            pred_encode = tf.matmul(decode_factor, w3) + b3
        return pred_encode


    #Inputs:
    #   pred_encode : tensor, [None, 1024]
    #   cnn_shapes : python list, shapes of each convolution layers
    #Outputs:
    #   pred : tensor, [None, 1, 33600]
    def conv_decoder(pred_encode, cnn_shapes):
        with tf.variable_scope("decoder"):
            with tf.variable_scope("dense_layers", reuse=tf.AUTO_REUSE):
                w = tf.get_variable("dense_weights", [1024, 14*10*64], initializer=tf.random_normal_initializer())
                b = tf.get_variable("dense_bias", [14*10*64], initializer=tf.random_normal_initializer())
                dense = tf.matmul(pred_encode, w) + b

                conv_trans_input = tf.reshape(dense, tf.stack([tf.shape(pred_encode)[0], cnn_shapes[4][1], cnn_shapes[4][2], cnn_shapes[4][3]]))

        with tf.variable_scope("cnn_transpose_layers", reuse=tf.AUTO_REUSE):
            w1 = tf.get_variable("conv_transpose1_weights", [4, 4, 64, 64], initializer=tf.random_normal_initializer())
            conv_transpose1 = tf.nn.relu(tf.nn.conv2d_transpose(conv_trans_input, w1,
                                                                tf.stack([tf.shape(pred_encode)[0], cnn_shapes[3][1], cnn_shapes[3][2], cnn_shapes[3][3]]),
                                                                strides=[1, 2, 2, 1], padding='SAME'))

            w2 = tf.get_variable("conv_transpose2_weights", [6, 6, 32, 64], initializer=tf.random_normal_initializer())
            conv_transpose2 = tf.nn.relu(tf.nn.conv2d_transpose(conv_transpose1, w2,
                                                                tf.stack([tf.shape(pred_encode)[0], cnn_shapes[2][1], cnn_shapes[2][2], cnn_shapes[2][3]]),
                                                                strides=[1, 2, 2, 1],padding='SAME'))

            w3 = tf.get_variable("conv_transpose3_weights", [6, 6, 16, 32], initializer=tf.random_normal_initializer())
            conv_transpose3 = tf.nn.relu(tf.nn.conv2d_transpose(conv_transpose2, w3, 
                                                                tf.stack([tf.shape(pred_encode)[0], cnn_shapes[1][1], cnn_shapes[1][2], cnn_shapes[1][3]]), 
                                                                strides=[1, 2, 2, 1],padding='SAME'))

            w4 = tf.get_variable("conv_transpose4_weights", [4, 4, 1, 16], initializer=tf.random_normal_initializer())
            conv_transpose4 = tf.nn.relu(tf.nn.conv2d_transpose(conv_transpose3, w4, 
                                                                tf.stack([tf.shape(pred_encode)[0], cnn_shapes[0][1], cnn_shapes[0][2], 1]), 
                                                                strides=[1, 2, 2, 1],padding='SAME'))
            NCHW_conv_transpose4 = tf.transpose(conv_transpose4, [0, 3, 1, 2])

        pred = tf.reshape(NCHW_conv_transpose4, tf.stack([tf.shape(NCHW_conv_transpose4)[0], 1, 33600]))
        return pred

In [17]:
class AEModel(object):
    def __init__(self, mean_img = np.zeros([1, 1, 33600])) :
        mean_img = np.reshape(mean_img, newshape=[1, 1, 33600])

        tf.reset_default_graph()
        self.x = tf.placeholder(tf.float32, [None, K, 33600], name='x')
        self.y = tf.placeholder(tf.float32, [None, NUM_STEP, 33600], name='y')
        self.n_step_acts = tf.placeholder(tf.float32, [None, NUM_STEP, 18], name='n_step_acts')
        self.one_step_act = tf.placeholder(tf.float32, [None, 18], name='one_step_act')

        self.mean = tf.Variable(mean_img, trainable=False, dtype=tf.float32)
        self.x_mean = (self.x-self.mean)/255

        self.train_nn()
        self.one_step_pred_nn()


    def predict(self, current_k_screens, current_act):
        #encode
        encode, conv_shapes = layer.conv_encoder(current_k_screens)

        #action_transform
        pred_encode = layer.action_transform(encode, current_act)

        #decode
        pred = layer.conv_decoder(pred_encode, conv_shapes)

        #next step input
        pred_mean = (pred-self.mean)/255
        #print(pred_mean.get_shape().as_list())
        ns_ksub1_indices = tf.constant([[i, k] for i in range(0, BATCH_SIZE) for k in range(1, K) ]) 
        ns_ksub1_screens = tf.gather_nd(current_k_screens, ns_ksub1_indices)
        ns_ksub1_screens = tf.reshape(ns_ksub1_screens, tf.stack([tf.shape(current_k_screens)[0], K-1, 33600]))
        ns_k_screens = tf.concat([ns_ksub1_screens, pred_mean], 1)
        return pred, ns_k_screens


    def train_nn(self):
        y_hat_list = []
        with tf.variable_scope("train_nn"):
            current_k_screens = self.x_mean
            for step in range(0, NUM_STEP):
                current_act_indices = tf.constant([[i, step] for i in range(BATCH_SIZE)]) 
                current_act = tf.gather_nd(self.n_step_acts, current_act_indices)  #act [None, 18]

                y_pred, next_k_screens = self.predict(current_k_screens, current_act)
                current_k_screens = next_k_screens
                y_hat_list.append(y_pred)

        with tf.variable_scope("cost"):
            y_hat = tf.concat(y_hat_list, axis = 1)
            self.cost = tf.reduce_mean(tf.square(y_hat - self.y), name="cost")

        with tf.variable_scope("optimize"):
            learning_rate = 0.001
            self.optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer").minimize(self.cost)


    #predict the next screen
    def one_step_pred_nn(self):
        current_k_screens = self.x_mean
        encode, conv_shapes = layer.conv_encoder(current_k_screens)
        pred_encode = layer.action_transform(encode, self.one_step_act)
        pred = layer.conv_decoder(pred_encode, conv_shapes)
        self.pred = tf.cast(pred, tf.int32, name = "pred")



In [18]:
#    raw data     #
#training data
n_train_screens = 2816
train_screens = np.zeros((n_train_screens, 33600))
train_screens_act = np.zeros((n_train_screens, NUM_ACTIONS))    #one hot actions, [0...17]
#develop data
n_dev_screens = 128
dev_screens = np.zeros((n_dev_screens, 33600))
dev_screens_act = np.zeros((n_dev_screens, NUM_ACTIONS))

# structured data #
#training data
train_x = np.zeros((n_train_screens, K, 33600))
train_y = np.zeros((n_train_screens, NUM_STEP, 33600))
train_act = np.zeros((n_train_screens, NUM_STEP, NUM_ACTIONS))
#develop data
dev_x = np.zeros((n_dev_screens, K, 33600))
dev_y = np.zeros((n_dev_screens, NUM_STEP, 33600))
dev_act = np.zeros((n_dev_screens, NUM_STEP, NUM_ACTIONS))


screen_dir = "./screens/alien/matrix_act/"
def loadScreen():
    print("load screens...")
    for i in range(0, n_train_screens):
        path = screen_dir + str(i + 1) + ".matrix"
        with open(path, "r") as f:
            data = f.read().split(' ')
            act = int(data[-2])
            train_screens_act[i][act] = 1

            pixels = data[:-2]
            pixels = list(map(int, pixels))
            train_screens[i] = np.array(pixels)

    for i in range(0, n_dev_screens):
        path = screen_dir + str(n_train_screens + i + 1) + ".matrix"
        with open(path, "r") as f:
            data = f.read().split(' ')

            act = int(data[-2])
            dev_screens_act[i][act] = 1 

            pixels = data[:-2]
            pixels = list(map(int, pixels))
            dev_screens[i] = np.array(pixels)
    print("screen loaded")


def buildData():
    print("build data...")
    for i in range(0, n_train_screens-T):
        train_x[i] = train_screens[i:i+K]
        train_y[i] = train_screens[i+K:i+T]
        train_act[i] = train_screens_act[i+K-1:i+T-1]

    for i in range(0, n_dev_screens-T):
        dev_x[i] = dev_screens[i:i+K]
        dev_y[i] = dev_screens[i+K:i+T]
        dev_act[i] = dev_screens_act[i+K-1:i+T-1]
    print("data builed")


n_batch = n_train_screens//batch_size
def train(mean_img):
    ae = AEModel(mean_img = mean_img)
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    py_saver = tf.train.Saver()
    c_saver = tf.train.Saver(tf.global_variables())

    writer = tf.summary.FileWriter("./AE_nn_log", sess.graph)

    n_epochs = 10
    for epoch_i in range(n_epochs):
        for batch_i in range(n_batch):
            #print(batch_i)
            batch_x = train_x[batch_i*batch_size : batch_i*batch_size + batch_size]
            batch_y = train_y[batch_i*batch_size : batch_i*batch_size + batch_size]
            batch_act = train_act[batch_i*batch_size : batch_i*batch_size + batch_size]
            sess.run(ae.optimizer, feed_dict={ae.x: batch_x, ae.y: batch_y, ae.n_step_acts: batch_act})
        print(epoch_i, sess.run(ae.cost, feed_dict={ae.x: dev_x, ae.y: dev_y, ae.n_step_acts: dev_act}))
    py_saver.save(sess, './ckpt/model')
    c_saver.save(sess, "c_ckpt/graph.ckpt")

    writer.close()
    sess.close()


if __name__ == '__main__':
    loadScreen()
    buildData()
    mean_img = np.mean(train_screens, 0)
    train(mean_img)


load screens...
screen loaded
build data...
data builed


InvalidArgumentError: Input to reshape is a tensor with 1075200 values, but the requested shape has 17203200
	 [[Node: train_nn/Reshape_2 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:CPU:0"](train_nn/GatherNd_1, train_nn/stack_2)]]

Caused by op 'train_nn/Reshape_2', defined at:
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-

In [5]:
import matplotlib.pyplot as plt

ae = AEModel()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(sess, "./ckpt/model")

screen_height = 210
screen_width = 160

n_train_screens = 2816

n_dev_screens = 128
dev_screens = np.zeros((n_dev_screens, 33600))
dev_screens_act = np.zeros((n_dev_screens, NUM_ACTIONS))

dev_x = np.zeros((n_dev_screens, K, 33600))
dev_y = np.zeros((n_dev_screens, NUM_STEP, 33600))
dev_act = np.zeros((n_dev_screens, NUM_STEP, NUM_ACTIONS))

screen_dir = "./screens/alien/matrix_act/"
def loadData(dir):
    for i in range(0, n_dev_screens):
        path = screen_dir + str(n_train_screens + i + 1) + ".matrix"
        with open(path, "r") as f:
            data = f.read().split(' ')

            act = int(data[-2])
            dev_screens_act[i][act] = 1 

            pixels = data[:-2]
            pixels = list(map(int, pixels))
            dev_screens[i] = np.array(pixels)
loadData(screen_dir)

recon = sess.run(ae.pred, feed_dict={ae.x: dev_x, ae.n_step_acts: dev_act[:, 0, :]})

n_examples = 2
fig, axs = plt.subplots(n_examples, 2, figsize=(210, 160), squeeze=False)
for example_i in range(n_examples):
    axs[example_i][0].imshow(
        np.reshape(dev_y[example_i], (210, 160)))
    axs[example_i][1].imshow(
        np.reshape(recon[example_i], (210, 160)))
fig.show()
plt.draw()

sess.close()

INFO:tensorflow:Restoring parameters from ./ckpt/model


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/figure.py:459: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "
